In [1]:
from torch_points3d.datasets.base_dataset import BaseDataset
from torch.utils.data import Dataset

In [2]:
import numpy as np
import pandas as pd
import time 
import os
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from torch_geometric.io import read_txt_array
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler

from plyfile import PlyData, PlyElement,PlyProperty, PlyListProperty

import sys
from omegaconf import OmegaConf
import pyvista as pv
import ipdb

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.max_colwidth", 256)

torch.set_printoptions(precision=8)

In [5]:
%matplotlib inline
sns.set(rc={'figure.figsize':(8,6)})


In [6]:
patient_id_list_new =  '2_BC	3_BC	4_BC	5_BM	6_BM	7_BP	8_BP	9_KBW	10_SUM	11_DHM	12_GAW	13_PMM	14_TR	15_TR	16_TR	18_EM	19_EM	20_EM	21_FA	22_FA	23_HJ	24_HJ	25_HM	26_HM	27_HM	28_JM	29_JM	30_JM	31_JM	32_JM	33_KBB	34_KBB	35_KBB	36_KBB	37_KBB	38_KBB	39_KBB	40_KBB	41_KBB	42_KBB	43_KBB	44_AC	45_AC	46_LE	47_LE	48_LE	50_LE	51_LE	52_LE	54_MR	55_MR	56_WA	57_SF	58_SI	59_SI'
patient_id_list_new = patient_id_list_new.split()
patient_id_list_new[:5]

['2_BC', '3_BC', '4_BC', '5_BM', '6_BM']

In [7]:
os.getcwd()

'D:\\Workspace\\Python\\torch-points3d\\notebooks'

In [8]:
DIR = os.getcwd() # Replace with your root directory, the data will go in DIR/data.

In [9]:
shapenet_yaml = """
class: shapenet.ShapeNetDataset
task: segmentation
dataroot: %s
first_subsampling: 0.02                       # Grid size of the input data
pre_transforms:                               # Offline transforms, done only once
    - transform: NormalizeScale           
    - transform: GridSampling3D
      params:
        size: ${first_subsampling}
train_transforms:                             # Data augmentation pipeline
    - transform: RandomNoise
      params:
        sigma: 0.01
        clip: 0.05
    - transform: RandomScaleAnisotropic
      params:
        scales: [0.9,1.1]
    - transform: AddOnes
    - transform: AddFeatsByKeys
      params:
        list_add_to_x: [True]
        feat_names: ["ones"]
        delete_feats: [True]
test_transforms:
    - transform: AddOnes
    - transform: AddFeatsByKeys
      params:
        list_add_to_x: [True]
        feat_names: ["ones"]
        delete_feats: [True]
""" % (os.path.join(DIR,"data")) 

from omegaconf import OmegaConf
params = OmegaConf.create(shapenet_yaml)
# if CATEGORY != "All":
#     params.category = CATEGORY

In [10]:
# params.raw_file_identifiers = patient_id_list_new
# params.raw_file_identifiers

In [10]:
SHUFFLED_SPLITS = {
    "train": ['2_BC', '3_BC', '4_BC', '5_BM', '6_BM', '7_BP', '9_KBW', '8_BP', '10_SUM', '11_DHM'],
    "val": ['12_GAW', '13_PMM', '14_TR', '15_TR', '16_TR', '18_EM'],
    "test": ['23_HJ', '24_HJ', '25_HM'],
}
params.shuffled_splits = SHUFFLED_SPLITS
params.shuffled_splits

{'train': ['2_BC', '3_BC', '4_BC', '5_BM', '6_BM', '7_BP', '9_KBW', '8_BP', '10_SUM', '11_DHM'], 'val': ['12_GAW', '13_PMM', '14_TR', '15_TR', '16_TR', '18_EM'], 'test': ['23_HJ', '24_HJ', '25_HM']}

In [11]:
category_to_seg = {'aneur': [0,1,2,3,4]
             }

In [12]:
from torch_points3d.datasets.segmentation import AneurysmDataset 
# dataset = AneurysmDataset.AneurysmDataset(params,category_to_seg )
dataset = AneurysmDataset(params,category_to_seg )
dataset

Processing...


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Done!


Dataset: AneurysmDataset 
train_pre_batch_collate_transform = None
val_pre_batch_collate_transform = None
test_pre_batch_collate_transform = None
pre_transform = Compose([
    NormalizeScale(),
    GridSampling3D(grid_size=0.02, quantize_coords=False, mode=mean),
])
test_transform = Compose([
    AddOnes(),
    AddFeatsByKeys(ones=True),
])
train_transform = Compose([
    RandomNoise(sigma=0.01, clip=0.05),
    RandomScaleAnisotropic([0.9, 1.1]),
    AddOnes(),
    AddFeatsByKeys(ones=True),
])
val_transform = None
inference_transform = Compose([
    NormalizeScale(),
    GridSampling3D(grid_size=0.02, quantize_coords=False, mode=mean),
    AddOnes(),
    AddFeatsByKeys(ones=True),
])
Size of train_dataset = 10
Size of test_dataset = 3
Size of val_dataset = 6
Batch size = None

In [13]:
dataset.test_dataset, dataset.train_dataset

([Aneurysm(3)], Aneurysm(10))

In [14]:
dataset.train_dataset[0]

Data(category=[7623], grid_size=[1], id_scan=[1], origin_id=[7623], pos=[7623, 3], x=[7623, 11], y=[7623])

In [15]:
dataset.test_dataset[0][0]

Data(category=[10063], grid_size=[1], id_scan=[1], origin_id=[10063], pos=[10063, 3], x=[10063, 11], y=[10063])

## Model for part segmentation
Let's start by creating a multihead segmentation module with one segmentation head per category. We provide that as part of Torch Points3D but let's reproduce it here for sake of completeness.

In [16]:
from torch_points3d.core.common_modules import MLP, UnaryConv

class MultiHeadClassifier(torch.nn.Module):
    """ Allows segregated segmentation in case the category of an object is known. 
    This is the case in ShapeNet for example.

    Parameters
    ----------
    in_features -
        size of the input channel
    cat_to_seg
        category to segment maps for example:
        {
            'Airplane': [0,1,2],
            'Table': [3,4]
        }

    """

    def __init__(self, in_features, cat_to_seg, dropout_proba=0.5, bn_momentum=0.1):
        super().__init__()
        self._cat_to_seg = {}
        self._num_categories = len(cat_to_seg) #2
        self._max_seg_count = 0
        self._max_seg = 0
        self._shifts = torch.zeros((self._num_categories,), dtype=torch.long)
        for i, seg in enumerate(cat_to_seg.values()): #2 times for 2 keys
            self._max_seg_count = max(self._max_seg_count, len(seg)) #overwriting every ith time
            self._max_seg = max(self._max_seg, max(seg))
            self._shifts[i] = min(seg)
            self._cat_to_seg[i] = seg

        self.channel_rasing = MLP(
            [in_features, self._num_categories * in_features], bn_momentum=bn_momentum, bias=False
        )
        if dropout_proba:
            self.channel_rasing.add_module("Dropout", torch.nn.Dropout(p=dropout_proba))

        self.classifier = UnaryConv((self._num_categories, in_features, self._max_seg_count))
        self._bias = torch.nn.Parameter(torch.zeros(self._max_seg_count,))

    def forward(self, features, category_labels, **kwargs):
        assert features.dim() == 2
        self._shifts = self._shifts.to(features.device)
        in_dim = features.shape[-1]
        features = self.channel_rasing(features)
        features = features.reshape((-1, self._num_categories, in_dim))
        features = features.transpose(0, 1)  # [num_categories, num_points, in_dim]
        features = self.classifier(features) + self._bias  # [num_categories, num_points, max_seg]
        ind = category_labels.unsqueeze(-1).repeat(1, 1, features.shape[-1]).long()

        logits = features.gather(0, ind).squeeze(0)
        softmax = torch.nn.functional.log_softmax(logits, dim=-1)

        output = torch.zeros(logits.shape[0], self._max_seg + 1).to(features.device)
        cats_in_batch = torch.unique(category_labels)
        for cat in cats_in_batch:
            cat_mask = category_labels == cat
            seg_indices = self._cat_to_seg[cat.item()]
            probs = softmax[cat_mask, : len(seg_indices)]
            output[cat_mask, seg_indices[0] : seg_indices[-1] + 1] = probs
        
        return output

The model we implement here follows the main architecture proposed in the [original paper](https://arxiv.org/abs/1904.08889):

<p align="center">
  <img width="70%" src="https://drive.google.com/uc?export=view&id=1CJppQ88T69whjYsJc016L3_E_rtcJ8n1" />
</p>

In [17]:
from torch_points3d.applications.kpconv import KPConv


class PartSegKPConv(torch.nn.Module):
    def __init__(self, cat_to_seg):
        super().__init__()
        self.unet = KPConv(
            architecture="unet", 
            input_nc=10, 
            num_layers=4, 
            in_grid_size=0.0002
            )
        self.classifier = MultiHeadClassifier(self.unet.output_nc, cat_to_seg)
    
    @property
    def conv_type(self):
        """ This is needed by the dataset to infer which batch collate should be used"""
        return self.unet.conv_type
    
    def get_batch(self):
        return self.batch
    
    def get_output(self):
        """ This is needed by the tracker to get access to the ouputs of the network"""
        return self.output
    
    def get_labels(self):
        """ Needed by the tracker in order to access ground truth labels"""
        return self.labels
    
    def get_current_losses(self):
        """ Entry point for the tracker to grab the loss """
        return {"loss_seg": float(self.loss_seg)}

    def forward(self, data):
        self.labels = data.y
        self.batch = data.batch
        
        # Forward through unet and classifier
        data_features = self.unet(data)
        self.output = self.classifier(data_features.x, data.category)

         # Set loss for the backward pass
        self.loss_seg = torch.nn.functional.nll_loss(self.output, self.labels)
        return self.output

    def get_spatial_ops(self):
        return self.unet.get_spatial_ops()
        
    def backward(self):
         self.loss_seg.backward() 

    

In [18]:
model = PartSegKPConv(dataset.class_to_segments)

## The data loaders and CPU pre computing features
KPConv is quite demanding on spatial operations such as grid sampling and radius search. On the network loaded here we have 10 KPConv layers on the encoder which means 10 radius search operations with varying number of neighbours. We observed a significant performance gain by moving those operations to the CPU where they can easily be optimised with suitable data structures such as kd-tree. We use [nonaflann](https://github.com/jlblancoc/nanoflann) in the back-end, a 3D optimised kd-tree implementation. Note that this is beneficiary only if you have access to multiple CPU threads.

You can decide to precompute those spatial operations by setting the `precompute_multi_scale` parameter to `True` when creating the data loaders. The dataset will mine the model to figure out which spatial operations are required and in which order.

In [19]:
NUM_WORKERS = 4
BATCH_SIZE = 3
dataset.create_dataloaders(
    model,
    batch_size=BATCH_SIZE, 
    num_workers=NUM_WORKERS, 
    shuffle=True, 
    precompute_multi_scale=True 
    )

In [20]:
sample = next(iter(dataset.train_dataloader))
sample.keys

['x',
 'y',
 'pos',
 'multiscale',
 'upsample',
 'batch',
 'category',
 'grid_size',
 'id_scan',
 'origin_id']

Our `sample` contains the pre computed spatial information in the `multiscale` (encoder side) and `upsample` (decoder) attrivutes. The decoder pre computing is quite simple and just involves some basic caching for the nearest neighbour interpolation operation. Let's take a look at the encoder side of things first. 

In [21]:
sample.batch

tensor([0, 0, 0,  ..., 2, 2, 2])

In [22]:
sample.multiscale

[Batch(batch=[27363], idx_neighboors=[27363, 25], pos=[27363, 3]),
 Batch(batch=[27363], idx_neighboors=[27363, 25], pos=[27363, 3]),
 Batch(batch=[27363], grid_size=[3], idx_neighboors=[27363, 25], pos=[27363, 3]),
 Batch(batch=[27363], idx_neighboors=[27363, 25], pos=[27363, 3]),
 Batch(batch=[27363], grid_size=[3], idx_neighboors=[27363, 25], pos=[27363, 3]),
 Batch(batch=[27363], idx_neighboors=[27363, 25], pos=[27363, 3]),
 Batch(batch=[27359], grid_size=[3], idx_neighboors=[27359, 25], pos=[27359, 3]),
 Batch(batch=[27359], idx_neighboors=[27359, 25], pos=[27359, 3]),
 Batch(batch=[27341], grid_size=[3], idx_neighboors=[27341, 25], pos=[27341, 3]),
 Batch(batch=[27341], idx_neighboors=[27341, 25], pos=[27341, 3])]

`sample.multiscale` contains 10 different versions of the input batch, each one of these versions contains the location of the points in `pos` as well as the neighbours of these points in the previous point cloud. We will first look at the points coming out of each downsampling layer (strided convolution), we have 5 of them.

## Training loop

In [23]:
from tqdm.auto import tqdm
import time

class Trainer:
    def __init__(self,model, dataset, num_epoch = 3, device=torch.device('cpu')):
        self.num_epoch = num_epoch
        self._model = model
        self._dataset=dataset
        self.device = device

    def fit(self):
        self.optimizer = torch.optim.Adam(self._model.parameters(), lr=0.001)
        self.tracker = self._dataset.get_tracker(False, True)

        for i in range(self.num_epoch):
            print("=========== EPOCH %i ===========" % i)
            time.sleep(0.5)
            self.train_epoch()
            self.tracker.publish(i)
            self.test_epoch()
            self.tracker.publish(i)

    def train_epoch(self):
        self._model.to(self.device)
        self._model.train()
        self.tracker.reset("train")
        train_loader = self._dataset.train_dataloader
        iter_data_time = time.time()
        with tqdm(train_loader) as tq_train_loader:
            for i, data in enumerate(tq_train_loader):
                t_data = time.time() - iter_data_time
                iter_start_time = time.time()
                self.optimizer.zero_grad()
                data.to(self.device)
                self._model.forward(data)
                self._model.backward()
                self.optimizer.step()
                if i % 10 == 0:
                    self.tracker.track(self._model)

                tq_train_loader.set_postfix(
                    **self.tracker.get_metrics(),
                    data_loading=float(t_data),
                    iteration=float(time.time() - iter_start_time),
                )
                iter_data_time = time.time()

    def test_epoch(self):
        self._model.to(self.device)
        self._model.eval()
        self.tracker.reset("test")
        test_loader = self._dataset.test_dataloaders[0]
        iter_data_time = time.time()
        with tqdm(test_loader) as tq_test_loader:
            for i, data in enumerate(tq_test_loader):
                t_data = time.time() - iter_data_time
                iter_start_time = time.time()
                data.to(self.device)
                self._model.forward(data)           
                self.tracker.track(self._model)

                tq_test_loader.set_postfix(
                    **self.tracker.get_metrics(),
                    data_loading=float(t_data),
                    iteration=float(time.time() - iter_start_time),
                )
                iter_data_time = time.time()

In [24]:
trainer = Trainer(model, dataset)

In [25]:
trainer.fit()

=========== EPOCH 0 ===========


  0%|          | 0/4 [00:18<?, ?it/s]

  0%|          | 0/1 [00:18<?, ?it/s]

=========== EPOCH 1 ===========


  0%|          | 0/4 [00:22<?, ?it/s]

  0%|          | 0/1 [00:22<?, ?it/s]

=========== EPOCH 2 ===========


  0%|          | 0/4 [00:21<?, ?it/s]

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:75] data. DefaultCPUAllocator: not enough memory: you tried to allocate 854016000 bytes. Buy new RAM!